#### A1: Research Question

The question that I will be asking is "Which combinations of telecom products and services are most frequently purchased together by customers at risk of churn?"

#### A2: Goal of the Data Analysis

The goal is to identify patterns in the combinations of products and services purchased by customers. By understanding these patterns, we can develop targeted marketing strategies and personalized offers to enhance customer retention.

#### B1: Market Basket Analysis Explained

Market Basket Analysis (MBA) is a data mining technique used to enhance marketing strategies by identifying relationships between items that customers buy together frequently. In this project, we apply the Apriori algorithm, a classic MBA method, to uncover these relationships within the telecommunications dataset.

The Apriori algorithm identifies frequent individual items in the dataset and extends them to larger and larger item sets as long as those item sets appear sufficiently often in the database. The key metrics used to express the strength of associations are:
* Support - which indicates how frequently the itemset appears in the data set.
* Confidence - which indicates the likelihood of item Y being purchased when item X is purchased.
* Lift - which indicates the likelihood of item Y being purchased when item X is purchased, while controlling for the popularity of Y.

This analysis will help us understand which combinations of products and services are most appealing to customers at risk of churn, thus informing targeted marketing strategies to enhance customer retention.


(365 Data Science)

#### B2: Example of Transactions in the Dataset

An example of a transaction from the dataset includes several items that suggest a technologically savvy customer with needs ranging from computing essentials to accessories for convenience and maintenance. Specifically, the customer purchased a "Logitech M510 Wireless Mouse," indicating the need for computer peripherals, "HP 63 Ink" and "HP 65 Ink," showing they own and use a printer frequently, and a "nonda USB C to USB Adapter," which is typically used for devices that require a USB-C connection. They also bought a "10ft iPhone Charger Cable," likely for ease of device charging over longer distances, and "Creative Pebble 2.0 Speakers," which are compact speakers that could be used for personal entertainment or as part of a home office setup. This transaction reflects a customer whose purchases are geared towards enhancing their digital and computing experience.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import association_rules, apriori
from mlxtend.preprocessing import TransactionEncoder

# Loading the dataset
df = pd.read_csv(r'C:\Users\Hien Ta\OneDrive\WGU\MSDA\D212\Task_3\teleco_market_basket.csv')


# Check data types, number of values and size of dataframe
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The dataset consists of transactions, each representing a customer's purchase history, with up to 20 items per transaction. Many transactions contain NaN values, indicating customers who purchased fewer than 20 items. For market basket analysis, we need to transform this dataset into a suitable format that lists each item per transaction as a separate entry.The dataset consists of transactions, each representing a customer's purchase history, with up to 20 items per transaction. Many transactions contain NaN values, indicating customers who purchased fewer than 20 items. For market basket analysis, we need to transform this dataset into a suitable format that lists each item per transaction as a separate entry.

#### B3: Summarize one assumption of market basket analysis

Market basket analysis operates under the assumption that the purchase of certain items together within a transaction is not random, but has an underlying pattern that can be discovered and leveraged. It assumes that if a group of items is frequently purchased together, it is because customers have a particular preference or need that leads them to buy these items in combination. This preference is expected to persist over time, allowing the company to predict future buying behaviors and tailor their marketing and sales strategies accordingly. The assumption is that uncovering these patterns can lead to better customer insights and more effective cross selling opportunities.

(365 Data Science)

#### C1 Data Transformation

To prepare the dataset for Market Basket Analysis (MBA), we first transform the raw transaction data into a format suitable for the Apriori algorithm. The transformation process includes:

Data Cleaning:
* Removal of `NaN` values which represent missing items in transactions.
* Exclusion of transactions that do not contain any items post-cleanup.

Data Encoding:
* Application of a `TransactionEncoder` to convert the list of transaction items into a one-hot encoded matrix. Each column corresponds to a product available in the dataset, and each row corresponds to a transaction where '1' indicates the presence of the item in that transaction, and '0' indicates its absence.

The cleaned and encoded data is essential for the effective application of the Apriori algorithm, ensuring that my analysis only includes valid and complete transactions.

In [2]:
# Check data types, number of values and size of dataframe
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15002 entries, 0 to 15001
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Item01  7501 non-null   object
 1   Item02  5747 non-null   object
 2   Item03  4389 non-null   object
 3   Item04  3345 non-null   object
 4   Item05  2529 non-null   object
 5   Item06  1864 non-null   object
 6   Item07  1369 non-null   object
 7   Item08  981 non-null    object
 8   Item09  654 non-null    object
 9   Item10  395 non-null    object
 10  Item11  256 non-null    object
 11  Item12  154 non-null    object
 12  Item13  87 non-null     object
 13  Item14  47 non-null     object
 14  Item15  25 non-null     object
 15  Item16  8 non-null      object
 16  Item17  4 non-null      object
 17  Item18  4 non-null      object
 18  Item19  3 non-null      object
 19  Item20  1 non-null      object
dtypes: object(20)
memory usage: 2.3+ MB


,Item01,Item02,Item03,Item04,Item05,Item06,Item07,Item08,Item09,Item10,Item11,Item12,Item13,Item14,Item15,Item16,Item17,Item18,Item19,Item20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Logitech M510 Wireless mouse,HP 63 Ink,HP 65 ink,nonda USB C to USB Adapter,10ft iPHone Charger Cable,HP 902XL ink,Creative Pebble 2.0 Speakers,Cleaning Gel Universal Dust Cleaner,Micro Center 32GB Memory card,YUNSONG 3pack 6ft Nylon Lightning Cable,TopMate C5 Laptop Cooler pad,Apple USB-C Charger cable,HyperX Cloud Stinger Headset,TONOR USB Gaming Microphone,Dust-Off Compressed Gas 2 pack,3A USB Type C Cable 3 pack 6FT,HOVAMP iPhone charger,SanDisk Ultra 128GB card,FEEL2NICE 5 pack 10ft Lighning cable,FEIYOLD Blue light Blocking Glasses
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple Lightning to Digital AV Adapter,TP-Link AC1750 Smart WiFi Router,Apple Pencil,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Transforming the dataset to a list of lists and removing NaN values
transactions = df.apply(lambda x: x.dropna().tolist(), axis=1).tolist()

# Remove empty transactions
transactions = [transaction for transaction in transactions if transaction]

# Convert the non-empty transactions list of lists into a DataFrame ti 
transactions_df = pd.DataFrame({'Transaction': [', '.join(transaction) for transaction in transactions]})

transactions_df

,Transaction
0,"Logitech M510 Wireless mouse, HP 63 Ink, HP 65..."
1,"Apple Lightning to Digital AV Adapter, TP-Link..."
2,UNEN Mfi Certified 5-pack Lightning Cable
3,"Cat8 Ethernet Cable, HP 65 ink"
4,"Dust-Off Compressed Gas 2 pack, Screen Mom Scr..."
...,...
7496,"SanDisk 32GB Ultra SDHC card, Vsco 70 pack sti..."
7497,"Apple Lightning to Digital AV Adapter, Nylon B..."
7498,Falcon Dust Off Compressed Gas
7499,"HP 63XL Ink, Apple USB-C Charger cable"


In [4]:
# Saving the cleaned dataset without empty transactions
cleaned_file_path = r'C:\Users\Hien Ta\OneDrive\WGU\MSDA\D212\Task_3\teleco_market_basket_CLEAN.csv'
transactions_df.to_csv(cleaned_file_path, index=False)

#### C2: Confidence of the Association Rule

In [5]:
# transactions_df is the DataFrame loaded from the CSV with a single 'Transaction' column
transactions_list = transactions_df['Transaction'].apply(lambda x: x.split(', ')).tolist()

encoder = TransactionEncoder()
transactions_encoded = encoder.fit_transform(transactions_list)
transactions_onehot = pd.DataFrame(transactions_encoded, columns=encoder.columns_)

transactions_onehot

,10ft iPHone Charger Cable,10ft iPHone Charger Cable 2 Pack,3 pack Nylon Braided Lightning Cable,3A USB Type C Cable 3 pack 6FT,5pack Nylon Braided USB C cables,ARRIS SURFboard SB8200 Cable Modem,Anker 2-in-1 USB Card Reader,Anker 4-port USB hub,Anker USB C to HDMI Adapter,Apple Lightning to Digital AV Adapter,...,hP 65 Tri-color ink,iFixit Pro Tech Toolkit,iPhone 11 case,iPhone 12 Charger cable,iPhone 12 Pro case,iPhone 12 case,iPhone Charger Cable Anker 6ft,iPhone SE case,nonda USB C to USB Adapter,seenda Wireless mouse
0,True,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,False,False,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7497,False,False,False,False,False,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
7498,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7499,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [6]:
# Apply the Apriori algorithm
frequent_itemsets = apriori(transactions_onehot, min_support=0.01, use_colnames=True)

frequent_itemsets

# (365 Data Science)

,support,itemsets
0,0.050527,(10ft iPHone Charger Cable 2 Pack)
1,0.042528,(3A USB Type C Cable 3 pack 6FT)
2,0.019064,(5pack Nylon Braided USB C cables)
3,0.010932,(ARRIS SURFboard SB8200 Cable Modem)
4,0.029463,(Anker 2-in-1 USB Card Reader)
...,...,...
252,0.017064,"(VIVO Dual LCD Monitor Desk mount, SanDisk Ult..."
253,0.015731,"(VIVO Dual LCD Monitor Desk mount, Dust-Off Co..."
254,0.011465,"(VIVO Dual LCD Monitor Desk mount, Stylus Pen ..."
255,0.010132,"(USB 2.0 Printer cable, VIVO Dual LCD Monitor ..."


In [7]:
# diplay association_rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

# print out the DataFrame containing the association rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


                                           antecedents  \
0                   (10ft iPHone Charger Cable 2 Pack)   
1                   (10ft iPHone Charger Cable 2 Pack)   
2                   (10ft iPHone Charger Cable 2 Pack)   
3                      (Screen Mom Screen Cleaner kit)   
4                   (10ft iPHone Charger Cable 2 Pack)   
..                                                 ...   
315  (USB 2.0 Printer cable, Dust-Off Compressed Ga...   
316  (VIVO Dual LCD Monitor Desk mount, Dust-Off Co...   
317      (HP 61 ink, VIVO Dual LCD Monitor Desk mount)   
318         (HP 61 ink, Screen Mom Screen Cleaner kit)   
319  (VIVO Dual LCD Monitor Desk mount, Screen Mom ...   

                            consequents   support  confidence      lift  
0      (Dust-Off Compressed Gas 2 pack)  0.023064    0.456464  1.914955  
1                           (HP 61 ink)  0.010132    0.200528  1.223888  
2       (Screen Mom Screen Cleaner kit)  0.015198    0.300792  2.321232  
3    (1

#### C3: values for the support, lift, and confidence of the association rules table

In [8]:
# see attached codes

# print out the DataFrame containing the association rules
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                                           antecedents  \
0                   (10ft iPHone Charger Cable 2 Pack)   
1                   (10ft iPHone Charger Cable 2 Pack)   
2                   (10ft iPHone Charger Cable 2 Pack)   
3                      (Screen Mom Screen Cleaner kit)   
4                   (10ft iPHone Charger Cable 2 Pack)   
..                                                 ...   
315  (USB 2.0 Printer cable, Dust-Off Compressed Ga...   
316  (VIVO Dual LCD Monitor Desk mount, Dust-Off Co...   
317      (HP 61 ink, VIVO Dual LCD Monitor Desk mount)   
318         (HP 61 ink, Screen Mom Screen Cleaner kit)   
319  (VIVO Dual LCD Monitor Desk mount, Screen Mom ...   

                            consequents   support  confidence      lift  
0      (Dust-Off Compressed Gas 2 pack)  0.023064    0.456464  1.914955  
1                           (HP 61 ink)  0.010132    0.200528  1.223888  
2       (Screen Mom Screen Cleaner kit)  0.015198    0.300792  2.321232  
3    (1

#### C4: Top 3 relavent rules

In [9]:
# see attached codes

top_three_rules = rules.sort_values(by='lift', ascending=False).head(3)
print(top_three_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

                                           antecedents  \
218               (SanDisk 128GB Ultra microSDXC card)   
219                          (SanDisk Ultra 64GB card)   
303  (VIVO Dual LCD Monitor Desk mount, Dust-Off Co...   

                              consequents   support  confidence      lift  
218             (SanDisk Ultra 64GB card)  0.015998    0.323450  3.291994  
219  (SanDisk 128GB Ultra microSDXC card)  0.015998    0.162822  3.291994  
303             (SanDisk Ultra 64GB card)  0.017064    0.285714  2.907928  


* Rule 1: People who purchase a SanDisk 128GB Ultra microSDXC card also tend to purchase a SanDisk Ultra 64GB card. The support for this rule is about 1.6%, which means that this combination of items appears in 1.6% of all transactions. The confidence of 32.35% indicates that there's about a one-third chance that someone buying the 128GB card will also buy the 64GB card. The lift value of approximately 3.22 suggests that these two items are over three times more likely to be bought together than each item individually.

* Rule 2: The reverse is also true; those who buy a SanDisk Ultra 64GB card often purchase a SanDisk 128GB Ultra microSDXC card. This has the same support of 1.6%, indicating it's just as common as the reverse. However, the confidence is lower at 16.28%, suggesting that the 64GB card is less influential in leading to the purchase of the 128GB card. The lift remains the same, reinforcing the strength of the association.

* Rule 3: This rule indicates that when customers buy both Dust-Off Compressed Gas 2 pack and VIVO Dual LCD Monitor Desk Mount, they are also likely to buy a SanDisk Ultra 64GB card. The support of approximately 1.71% suggests that these three items appear together in roughly 1.71% of all transactions. The confidence of about 28.57% implies that there's about a 28.57% chance that the 64GB SanDisk card will be bought when the other two items are purchased together. The lift of roughly 2.91 indicates that the presence of the gas pack and monitor desk mount in a transaction is about three times more likely to lead to the purchase of the 64GB SanDisk card than if the purchases were independent.

These rules can be quite insightful for the retailer or the marketing team. The first two rules suggest that customers are perhaps upgrading their storage options or buying for multiple devices with different storage needs. The third rule may point towards a demographic that is interested in maintaining and setting up tech products, indicating a crossover in the market for electronics maintenance and office ergonomics.

#### D1: Summarize the Significance of Support, Lift, and Confidence

* Support is the measure of the prevalence of an item or itemset within all transactions. A higher support value indicates that items are more common in the dataset. In market basket analysis, rules with higher support are seen as more significant as they affect a larger portion of the transactions.

* Confidence is an indication of how often the rule has been found to be true. In other words, it's the likelihood that the consequent is purchased when the antecedent is purchased. A high confidence value for a rule signifies a strong predictive power or reliability but does not account for the base popularity of the consequent item.

* Lift compares the observed frequency of A and B occurring together with the frequency that would be expected if A and B were independent. A lift value greater than 1 means that the items are more likely to be bought together. A lift less than 1 would indicate that items are less likely to be bought together. It’s a direct measure of the strength of an association.

(365 Data Science)

In [10]:
rules.support.value_counts()

support
0.017064    11
0.010532    11
0.011065    11
0.010932    10
0.011998    10
            ..
0.016264     1
0.017198     1
0.014931     1
0.022797     1
0.014665     1
Name: count, Length: 95, dtype: int64

In [11]:
rules.confidence.value_counts()

confidence
0.170576    2
0.108025    2
0.120920    2
0.110799    2
0.181237    2
           ..
0.348000    1
0.215126    1
0.343964    1
0.220917    1
0.308271    1
Name: count, Length: 306, dtype: int64

In [12]:
rules.lift.value_counts()

lift
1.522468    2
1.903546    2
1.553774    2
1.322437    2
1.132539    2
           ..
1.747522    1
1.421397    1
1.421397    1
1.630358    1
1.881480    1
Name: count, Length: 251, dtype: int64

#### D2: Discuss the Practical Significance of Findings.

The findings from the market basket analysis can provide valuable insights into customer purchasing patterns. For instance, the association between different storage capacities of SD cards could imply that customers who buy one type are likely to need or consider another type—possibly indicating multiple device ownership or different use cases. The combination of Dust-Off Compressed Gas with a VIVO Dual LCD Monitor Desk Mount suggests that a certain customer segment is interested in both maintaining their electronic devices and setting up an ergonomic workspace.

This information can inform strategies for product placement, promotions, and inventory management. For example, products that are frequently bought together could be placed closer to each other in the store or bundled together in promotions to increase the average transaction value. Moreover, knowing which items have strong associations can help in forecasting demand more accurately.

#### D3: Recommendations

Leveraging insights from my market basket analysis. The recommended course of action focuses on targeted marketing campaigns, product bundling, optimized placement, and personalized recommendations to address customer churn. By offering promotions on items frequently purchased together, such as bundling storage devices or combining electronics maintenance products with ergonomic solutions, the company can enhance perceived value and customer engagement. Additionally, arranging related products closely in store or online, along with providing personalized recommendations based on purchase history, can further reduce churn. Continuous customer feedback and employee training on these insights will ensure strategies remain effective and customer-centric model, ultimately improving retention and satisfaction.

#### E: Panopto Video

https://wgu.hosted.panopto.com/Panopto/Pages/Viewer.aspx?id=156813e9-b033-47b9-8b69-b14f01627b1f

#### F: Third-party Code

(365 Data Science). https://365datascience.com/tutorials/python-tutorials/market-basket-analysis/

#### G: Source

(365 Data Science). https://365datascience.com/tutorials/python-tutorials/market-basket-analysis/